
# Observabilidade e Monitoramento de Modelos com Evidently AI (Iris)

Este notebook implementa um **mini-case prático** de observabilidade e monitoramento de um modelo de classificação
utilizando o dataset **Iris**, simulando:
- **Data drift** (mudança na distribuição dos dados de entrada);
- **Degradação de performance** do modelo ao longo do tempo;
- **Relatórios** do [Evidently AI](https://docs.evidentlyai.com/) para _Data Drift_ e _Classification Performance_.

> **Como usar**: Execute célula a célula. É necessário ter internet para instalar o `evidently` caso não esteja no seu ambiente.


## 1. Setup do ambiente

In [17]:

# Se precisar, descomente para atualizar pip
# !pip install --upgrade pip

# Tentativa de importar; se falhar, instala pacotes necessários
def ensure(package):
    try:
        __import__(package)
        print(f"OK: {package} já instalado.")
    except ModuleNotFoundError:
        # Observação: requer internet para instalar
        import sys, subprocess
        print(f"Instalando {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Pacotes essenciais
ensure("pandas")
ensure("numpy")
ensure("sklearn")
# Evidently pode não estar instalado por padrão
try:
    import evidently
    print("OK: evidently já instalado.")
except ModuleNotFoundError:
    import sys, subprocess
    print("Instalando evidently...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "evidently"])

import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from pathlib import Path

# Imports Evidently
from evidently import Report, Dataset, DataDefinition, MulticlassClassification
from evidently.presets import DataDriftPreset
from evidently.presets import ClassificationPreset
print("Imports concluídos.")


OK: pandas já instalado.
OK: numpy já instalado.
OK: sklearn já instalado.
OK: evidently já instalado.
Imports concluídos.


## 2. Carregar dados (Iris) e preparar *baseline* vs *current*

In [3]:

iris = load_iris(as_frame=True)
df = iris.frame.copy()
df.rename(columns={c: c.replace(' (cm)', '').replace(' ', '_') for c in df.columns}, inplace=True)
df.head()


,sepal_length,sepal_width,petal_length,petal_width,target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [4]:

# Vamos separar um baseline (referência) e um conjunto "current" (dados mais recentes em produção)
# Usaremos uma partição simples. Em produção, baseline = treino/validação inicial; current = dados de produção recentes.
baseline, current = train_test_split(df, test_size=0.5, random_state=42, stratify=df['target'])

baseline = baseline.reset_index(drop=True)
current = current.reset_index(drop=True)

print("Baseline shape:", baseline.shape)
print("Current  shape:", current.shape)
baseline.head(3), current.head(3)


Baseline shape: (75, 5)
Current  shape: (75, 5)


(   sepal_length  sepal_width  petal_length  petal_width  target
 0           5.1          3.8           1.5          0.3       0
 1           5.0          2.0           3.5          1.0       1
 2           6.9          3.2           5.7          2.3       2,
    sepal_length  sepal_width  petal_length  petal_width  target
 0           5.1          3.7           1.5          0.4       0
 1           5.0          3.4           1.5          0.2       0
 2           5.0          3.2           1.2          0.2       0)

## 3. Treinar um modelo simples (Random Forest) no *baseline*

In [5]:

FEATURES = [c for c in df.columns if c != 'target']
TARGET = 'target'

X_base = baseline[FEATURES].values
y_base = baseline[TARGET].values

X_curr = current[FEATURES].values
y_curr = current[TARGET].values

model = RandomForestClassifier(n_estimators=200, random_state=42)
model.fit(X_base, y_base)

# Avaliar rapidamente no baseline e no current (antes de simular drift)
pred_base = model.predict(X_base)
pred_curr = model.predict(X_curr)

acc_base = accuracy_score(y_base, pred_base)
acc_curr = accuracy_score(y_curr, pred_curr)

print(f"Acurácia (baseline) = {acc_base:.3f}")
print(f"Acurácia (current)  = {acc_curr:.3f}")


Acurácia (baseline) = 1.000
Acurácia (current)  = 0.893


## 4. Simular **data drift** e **degradação de performance** no *current*

In [6]:

# Vamos criar uma versão "driftada" do conjunto current:
# Estratégias (exemplos simples):
# - deslocar/escala uma ou mais features (ex.: multiplicar 'petal_length' por 1.3)
# - adicionar ruído a uma feature
# - alterar levemente a distribuição (embaralhar parte dos valores de uma coluna)
current_drifted = current.copy()

# Exemplo: aumentar 30% os valores de 'petal_length' e adicionar ruído gaussiano em 'sepal_width'
if 'petal_length' in current_drifted.columns:
    current_drifted['petal_length'] = current_drifted['petal_length'] * 1.3

if 'sepal_width' in current_drifted.columns:
    rng = np.random.default_rng(42)
    current_drifted['sepal_width'] = current_drifted['sepal_width'] + rng.normal(0, 0.2, size=len(current_drifted))

# Prever novamente com o mesmo modelo para ver a performance após "drift"
pred_curr_drifted = model.predict(current_drifted[FEATURES].values)
acc_curr_drifted = accuracy_score(current_drifted[TARGET].values, pred_curr_drifted)

print(f"Acurácia (baseline)        = {acc_base:.3f}")
print(f"Acurácia (current original)= {acc_curr:.3f}")
print(f"Acurácia (current drifted) = {acc_curr_drifted:.3f}")


Acurácia (baseline)        = 1.000
Acurácia (current original)= 0.893
Acurácia (current drifted) = 0.760


## 5. Preparar *dataframes* com predições para os relatórios do Evidently

In [7]:

# O Evidently espera termos colunas de 'target' e 'prediction'. Vamos construir tais colunas.
baseline_for_eval = baseline.copy()
current_for_eval  = current_drifted.copy()

baseline_for_eval['prediction'] = pred_base
current_for_eval['prediction']  = pred_curr_drifted

baseline_for_eval.head(3), current_for_eval.head(3)


(   sepal_length  sepal_width  petal_length  petal_width  target  prediction
 0           5.1          3.8           1.5          0.3       0           0
 1           5.0          2.0           3.5          1.0       1           1
 2           6.9          3.2           5.7          2.3       2           2,
    sepal_length  sepal_width  petal_length  petal_width  target  prediction
 0           5.1     3.760943          1.95          0.4       0           0
 1           5.0     3.192003          1.95          0.2       0           0
 2           5.0     3.350090          1.56          0.2       0           0)

## 6. Relatório de **Data Drift** (Evidently)

In [ ]:
# Cria uma pasta "reports" no mesmo diretório do notebook (se não existir)
reports_dir = Path("reports")
reports_dir.mkdir(exist_ok=True)

print(f"Relatórios serão salvos em: {reports_dir.resolve()}")


Relatórios serão salvos em: C:\Users\anacl\Downloads\reports


In [12]:

# Define apenas colunas numéricas/categóricas
data_def_drift = DataDefinition(
    numerical_columns=FEATURES,
    categorical_columns=[],
)

ref_ds_drift = Dataset.from_pandas(baseline[FEATURES], data_definition=data_def_drift)
cur_ds_drift = Dataset.from_pandas(current_drifted[FEATURES], data_definition=data_def_drift)

drift_report = Report([DataDriftPreset()])
drift_result = drift_report.run(current_data=cur_ds_drift, reference_data=ref_ds_drift)

drift_html = reports_dir / "iris_data_drift_report.html"
drift_result.save_html(str(drift_html))
print(f"Relatório salvo em: {drift_html.resolve()}")


Relatório salvo em: C:\Users\anacl\Downloads\reports\iris_data_drift_report.html


## 7. Relatório de **Performance de Classificação** (Evidently)

In [18]:
# (garanta que existam colunas 'target' e 'prediction' no DF de avaliação)
baseline_for_eval = baseline_for_eval.copy()
current_for_eval  = current_for_eval.copy()
baseline_for_eval["target"] = baseline_for_eval["target"].astype(str)
current_for_eval["target"]  = current_for_eval["target"].astype(str)
baseline_for_eval["prediction"] = baseline_for_eval["prediction"].astype(str)
current_for_eval["prediction"]  = current_for_eval["prediction"].astype(str)

# mapeie tipos e papéis das colunas (features numéricas + papéis de classificação)
num_cols = [c for c in baseline_for_eval.columns if c not in ("target", "prediction")]
data_def = DataDefinition(
    numerical_columns=num_cols,
    # para Iris (multiclasse): informe onde está o alvo e o rótulo previsto
    classification=[MulticlassClassification(
        target="target",
        prediction_labels="prediction",
        # se tivesse probabilidades em colunas, passar prediction_probas=["0","1","2"]
    )]
)

# crie os Datasets com a mesma definição
ref_ds = Dataset.from_pandas(baseline_for_eval, data_definition=data_def)
cur_ds = Dataset.from_pandas(current_for_eval,  data_definition=data_def)

# gere e execute o Report
perf_report = Report([ClassificationPreset()])
perf_result = perf_report.run(current_data=cur_ds, reference_data=ref_ds)

# salve
from pathlib import Path
reports_dir = Path("reports"); reports_dir.mkdir(exist_ok=True)
perf_html = reports_dir / "iris_classification_performance_report.html"
perf_result.save_html(str(perf_html))
print(f"Relatório salvo em: {perf_html.resolve()}")

Relatório salvo em: C:\Users\anacl\Downloads\reports\iris_classification_performance_report.html



## 8. Interpretando os resultados
Abra os arquivos HTML gerados na pasta `reports/`:

- `iris_data_drift_report.html`: compara a distribuição das *features* entre **baseline** e **current**.
  - Procure pelo **Overall Dataset Drift** (percentual de colunas com drift) e pelas colunas sinalizadas com drift significativo.
  - Inspecione os gráficos de distribuição para entender **como** mudaram.

- `iris_classification_performance_report.html`: compara as métricas de **classificação** entre baseline e current.
  - Observe a **acurácia**, **precision/recall por classe** e a **matriz de confusão**.
  - Verifique se houve **degradação** e em quais classes (espécies) ela foi mais evidente.

> Dica: conecte os dois relatórios. Se houve queda de performance, **qual drift nos dados pode explicar?** Ex.: após aumentar `petal_length` artificialmente, o modelo pode confundir espécies que dependiam fortemente dessa feature.


## 9. (Opcional) Limiares e alertas

In [20]:
# Exemplo simples: defina limiares e tome decisões automáticas
DRIFT_THRESHOLD_PCT = 0.3   # se 30%+ das colunas tiverem drift, acionar ação
ACC_DROP_THRESHOLD  = 0.10  # se queda de acurácia >= 10 pp, acionar ação

# Atenção: substitua pelos valores reais extraídos do objeto drift_result/perf_result
pct_columns_drifted = 0.25   # exemplo didático
acc_base_val = 0.95
acc_curr_val = 0.82
acc_drop = acc_base_val - acc_curr_val

print(">> Este bloco é ilustrativo. Para extrair % de colunas com drift do Evidently, "
      "você pode inspecionar o objeto 'drift_result' em JSON ou usar APIs do Evidently.\n"
      "Aqui mostramos como você pode implementar lógica de decisão no seu pipeline.\n")

print("Exemplo de decisão:")
if pct_columns_drifted >= DRIFT_THRESHOLD_PCT:
    print(f"- {pct_columns_drifted:.0%} das colunas com drift → acionar re-treino.")
if acc_drop >= ACC_DROP_THRESHOLD:
    print(f"- Queda de acurácia {acc_drop:.1%} ≥ {ACC_DROP_THRESHOLD:.0%} → abrir incidente/re-treino.")
if pct_columns_drifted < DRIFT_THRESHOLD_PCT and acc_drop < ACC_DROP_THRESHOLD:
    print("- Sem ações imediatas, apenas continuar monitorando.")


>> Este bloco é ilustrativo. Para extrair % de colunas com drift do Evidently, você pode inspecionar o objeto 'drift_result' em JSON ou usar APIs do Evidently.
Aqui mostramos como você pode implementar lógica de decisão no seu pipeline.

Exemplo de decisão:
- Queda de acurácia 13.0% ≥ 10% → abrir incidente/re-treino.



## 10. Boas práticas para produção (checklist rápido)
- **Log estruturado** de cada inferência (features essenciais, timestamp, versão do modelo, predição).
- **Coleta de *ground truth*** sempre que possível (ex.: feedback do usuário, eventos de negócio).
- **Painéis** (dashboards) com métricas de sistema (latência, erro) e de modelo (drift, acurácia, viés).
- **Limiares e alertas** definidos e acordados entre times (DS, Eng., Produto/Negócio, Governança).
- **Ciclo de re-treino** automatizado (ou semiautomatizado) com **validação offline** e **canary/A-B** em produção.
- **Explainability**: monitore mudanças na importância das features e explique decisões críticas.
- **Versionamento** de modelos e dados: rastreie o que estava em produção quando algo aconteceu.



## 11. Próximos passos
- Trocar o dataset para um caso de negócio seu (ex.: crédito, churn) e repetir o fluxo.
- Integrar a geração de relatórios do Evidently num **pipeline diário/semanal** (ex.: Airflow, GitHub Actions, Azure ML Pipelines).
- Exportar métricas do Evidently para um **Prometheus/Grafana/Datadog** e criar **alertas**.
- Adicionar **avaliações por segmento** (ex.: performance por região/faixa etária) para capturar problemas localizados.
- Incluir **testes de qualidade de dados** antes da inferência (valores faltantes, ranges, tipagem).
